<a href="https://colab.research.google.com/github/leshabirukov/categories-for-programmers/blob/master/mnist_my_sample_transform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import numpy as np
import matplotlib.pyplot as plt

import matplotlib as mpl

In [0]:
lRate   = 1.0
lGamma  = 0.7
lEpochs = 3
log_interval = 100
batch_size = 1000

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d( 1,  32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [0]:
class RandomNoise(object):
    def __init__(self, power = 0.0 ):
        self.power = power

    def __call__(self, sample):
        sample = sample * (1.0 - self.power)

        return sample

In [0]:
device = torch.device("cuda")

compTransform = transforms.Compose([RandomNoise(0.2),
                               transforms.ToTensor()])

train_dataset = datasets.MNIST(root='./data',
                           train=True,
                           transform=compTransform,
                           download=True)

test_dataset = datasets.MNIST(root='./data',
                           train=False,
                           transform=compTransform)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=64,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=1000,
                                          shuffle=False)

In [0]:
model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=lRate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=lGamma )

In [0]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:
for epoch in range(1, lEpochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

TypeError: ignored

In [0]:
model = model.to("cpu")
(smplStd,ansStd) = next(iter(test_loader))
lst        = F.nll_loss( model(smplStd), ansStd, reduction='none' )
bads = smplStd[ torch.nonzero( lst>1 ) ]
bads = bads.reshape( (-1, 1, 28, 28) )
bedA = model(bads.reshape(-1,1,28,28))
abc = torch.argmax(bedA, dim=1)
for j in range (2):
    for i in range (8):
        plt.subplot(2,8, i +j*8+1)
        plt.imshow( bads[i +j*8].reshape(28,28), cmap="gray_r")
        bedaE = enumerate( bedA[i +j*8] )
        print( ansStd[ torch.nonzero( lst>1 ) ][i +j*8], abc[i +j*8] ) 